# Trabalho 3
Descrição do trabalho:

http://webserver2.tecgraf.puc-rio.br/~mgattass/visao/trb/T3.html

Aluno: Daniel da Silva Costa

E-mail: danieldasilvacosta@gmail.com

### Imports

In [1]:
import numpy as np

### Usando o Numpy

#### Funções de ativação

In [2]:
def tanh(x):
  return np.tanh(x)

def tanh_prime(x):
  return 1-np.tanh(x)**2

In [3]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_prime(x):
  s = 1/(1+np.exp(-x))
  ds = s*(1-s)
  return ds

In [4]:
def relu(x):
  return np.maximum(0,x)

def relu_prime(x):
  return (x>0).astype(x.dtype)

#### Função loss

In [5]:
# loss function and its derivative
def mse(y_true, y_pred):
  return np.mean(np.power(y_true-y_pred, 2))

In [6]:
def mse_prime(y_true, y_pred):
  grad_output = 2*(y_pred-y_true)/y_true.size
  return grad_output

#### class Layer

In [7]:
# Base class
class Layer:
  def __init__(self):
    self.input = None
    self.output = None

  # computes the output Y of a layer for a given input X
  def forward(self, input):
    raise NotImplementedError

  # computes dE/dX for a given dE/dY (and update parameters, if any)
  def backward(self, grad_output, learning_rate):
    raise NotImplementedError

#### class FCLayer(Layer)

In [8]:
# inherit from base class Layer
class FCLayer(Layer):
  # input_size = number of input neurons
  # output_size = number of output neurons
  def __init__(self, input_size, output_size):
    self.weights = 2*np.random.rand(input_size, output_size) - 1.0
    self.bias = 2*np.random.rand(1, output_size) - 1.0

  # returns output for a given input
  def forward(self, input):
    self.input = input
    self.output = np.dot(input, self.weights) + self.bias
    return self.output

  # computes dE/dW, dE/dB for a given grad_output=dE/dY.
  # returns grad_input=dE/dX.
  # ajusta os pesos e bias em função da regra do
  # gradiente descendente
  def backward(self, grad_output, learning_rate):
    grad_input = np.dot(grad_output, self.weights.T)
    grad_weights = np.dot(self.input.T, grad_output)
    grad_bias = grad_output

    # update parameters
    self.weights -= learning_rate * grad_weights
    self.bias -= learning_rate * grad_bias
    return grad_input

#### class ActivationLayer(Layer)

In [9]:
class ActivationLayer(Layer):
  def __init__(self, activation, activation_prime):
    self.activation = activation
    self.activation_prime = activation_prime

  # returns the activated input
  def forward(self, input):
    self.input = input
    self.output = self.activation(self.input)
    return self.output

  # Returns grad_input=dE/dX given grad_output=dE/dY.
  # learning_rate is not used because there is no "learnable" parameters.
  def backward(self, grad_output, learning_rate):
    return self.activation_prime(self.input) * grad_output

#### class Network

In [10]:
class Network:
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_prime = None

  # add layer to network
  def add(self, layer):
    self.layers.append(layer)

  # set loss to use
  def use(self, loss, loss_prime):
    self.loss = loss
    self.loss_prime = loss_prime

  # predict output for given input
  def predict(self, input_data):
    # sample dimension first
    samples = len(input_data)
    result = []
    
    # run network over all samples
    for i in range(samples):
    
      # forward propagation
      output = input_data[i]
      for layer in self.layers:
        output = layer.forward_propagation(output)
        result.append(output)
    return result

  # train the network
  def fit(self, x_train, y_train, epochs, learning_rate):
    n_samples = x_train.shape[0]
    
    # training loop
    for i in range(epochs):
      err = 0
      for j in range(n_samples):

        # forward propagation
        output = x_train[j]
        for layer in self.layers:
          output = layer.forward (output)

        # compute loss (for display purpose only)
        err += self.loss(y_train[j], output)

        # backward propagation
        grad_out = self.loss_prime(y_train[j], output)
        for layer in reversed(self.layers):
          grad_out = layer.backward(grad_out, learning_rate)

      # calculate average error on all samples
      err /= n_samples
      print('epoch {i+1}/{epochs} error={err:.3f}')

#### Treinamento

In [11]:
x_train = np.array( [1, 2, 3, 4, 5] )
y_train = np.array( [2, 4, 6, 8, 10] )

In [12]:
network = Network()
input_size = len(x_train)
output_size = 7 # output_size = m = number of weights
network.add( FCLayer(input_size, output_size) )
network.add( ActivationLayer(sigmoid, sigmoid_prime) )
network.use( mse, mse_prime )
network.fit(x_train, 
            y_train, 
            epochs = 10, 
            learning_rate = 0.001)

ValueError: ignored